In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading all the libraries

In [11]:
# keras imports for the dataset and building our neural network
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model
# to calculate accuracy
from sklearn.metrics import accuracy_score
from __future__ import print_function
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from tensorflow.keras.layers import Conv2D,MaxPooling2D
#from IPython.display import display
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import os
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow import keras
from tensorflow.keras import layers

Defining the model

In [6]:
model = Sequential()
# convolutional layer
model.add(Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(1,1)))
# flatten output of conv
model.add(Flatten())
# hidden layer
model.add(Dense(100, activation='relu'))
# output layer
model.add(Dense(10, activation='softmax'))

Compiling the model and creating the data generator for training data.
Checkpointer is created using Accuracy as the monitor.

In [7]:
#Model is compiled
model.compile(optimizer = RMSprop(lr=0.0001),
               loss = 'categorical_crossentropy',
               metrics = ['accuracy'])

#Used to generate more images from training data
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
train_dir="/content/drive/My Drive/mnistTask"

#Loading and Preprocessing the training images
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    color_mode='grayscale',
                                                    class_mode = 'categorical',
                                                    target_size = (28,28))

#Checkpointer is created 
checkpointer = keras.callbacks.ModelCheckpoint(filepath = '/content/drive/My Drive/mnistTask/checkpoint-random-initialisation.hdf5',
                                               monitor='accuracy',
                                               mode='min',
                                               save_best_only=False,
                                               verbose=1)

Found 60000 images belonging to 10 classes.


In [8]:
epochs=20
history=model.fit(                                   #Model is fitted on the training data
                train_generator,
                steps_per_epoch=20,
                epochs=epochs,
                callbacks = [checkpointer])

Epoch 1/20
20/20 [==============================] - 175s 7s/step - loss: 2.2891 - accuracy: 0.1165

Epoch 00001: saving model to /content/drive/My Drive/mnistTask/checkpoint-random-initialisation.hdf5
Epoch 2/20
20/20 [==============================] - 155s 8s/step - loss: 2.2981 - accuracy: 0.1048

Epoch 00002: saving model to /content/drive/My Drive/mnistTask/checkpoint-random-initialisation.hdf5
Epoch 3/20
20/20 [==============================] - 152s 8s/step - loss: 2.3047 - accuracy: 0.0996

Epoch 00003: saving model to /content/drive/My Drive/mnistTask/checkpoint-random-initialisation.hdf5
Epoch 4/20
20/20 [==============================] - 149s 7s/step - loss: 2.3005 - accuracy: 0.0977

Epoch 00004: saving model to /content/drive/My Drive/mnistTask/checkpoint-random-initialisation.hdf5
Epoch 5/20
20/20 [==============================] - 151s 8s/step - loss: 2.3121 - accuracy: 0.0963

Epoch 00005: saving model to /content/drive/My Drive/mnistTask/checkpoint-random-initialisation.

Accuracy after 20 epochs - 0.1062

# ***Testing the model on mnist test set***

Loading the libraries

In [12]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as k
import matplotlib.pyplot as plt
import numpy as np

Loading the randomly initialised model 

In [13]:
trained_model =load_model('/content/drive/My Drive/mnistTask/checkpoint-random-initialisation.hdf5')

Loading the mnist dataset and preprocessing it

In [14]:
# loading the dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# building the input vector from the 28x28 pixels
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# normalizing the data to help with the training
X_train /= 255
X_test /= 255

# one-hot encoding using keras' numpy-related utilities
n_classes = 10
print("Shape before one-hot encoding: ", y_train.shape)
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)
print("Shape after one-hot encoding: ", Y_train.shape)

Shape before one-hot encoding:  (60000,)
Shape after one-hot encoding:  (60000, 10)


Reshaping the data so that it can be used with the model

In [15]:
#input image size 28*28
img_rows , img_cols = 28, 28
#reshaping
#this assumes our data format
#For 3D data, "channels_last" assumes (conv_dim1, conv_dim2, conv_dim3, channels) while 
#"channels_first" assumes (channels, conv_dim1, conv_dim2, conv_dim3).
if k.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
#more reshaping
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [16]:
num_category = 10
y_train = keras.utils.to_categorical(y_train, num_category)
y_test = keras.utils.to_categorical(y_test, num_category)
y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

Evaluating the accuracy on test set

In [17]:
score = trained_model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0]) #Test loss: 2.302675247192383
print('Test accuracy:', score[1]) #Test accuracy: 0.09799999743700027

Test loss: 2.302675247192383
Test accuracy: 0.09799999743700027
